In [1]:
#Train, val, Test splitter
import train_pytorch
import prepare_dataset
import random
import time 
from collections import Counter
import os
import numpy as np
import shutil
#os.chdir("/Users/kjartan/Documents/CompCogSci3/") 
os.chdir(r"C:\Users\kjart\OneDrive\Dokumenter\KU\3. semester\Cog sci 3\CompCogSci3")

In [2]:
language = "CN"
unsorted_data_dir = "raw_data/derivatives/"
annotation_file = f"raw_data/annotation/{language}/lpp{language}_word_information.csv"
random.seed(1234)

pos = "VERB" #Change this to select labels
oov = "-1" #no need to change this
binary = False #ignore this


In [ ]:
#Run this cell to load the data with the selected configuration, only need to run this cell if 
#you haven't loaded the data before, or if you want to change config

#Uncomment these two if you are changing config
#prepare_dataset.clear_data_dir()
#shutil.rmtree("aux_data/")

#prepare_dataset.load_data(unsorted_data_dir, language=language)
config_version = prepare_dataset.config2("data/", split=(0.8,0.1,0.1), language=language)
#config_version = prepare_dataset.config1("data/", split=(0.8,0.1,0.1), language=languge)

In [ ]:
#Only need to run this after an experiment if you want to change "pos", then run all the following cells

#Move data back from aux_data/
for phase in os.listdir("aux_data/"):
    for run in os.listdir(f"aux_data/{phase}"):
        path = f"aux_data/{phase}/{run}/{language}/"
        for file in os.listdir(path):
            file_path = f"data/{phase}/{run}/{language}/{file}"
            shutil.move(f"aux_data/{phase}/{run}/{language}/{file}",file_path)


In [ ]:


prepare_dataset.prepare_labels(annotation_file, "data/", language, pos=pos, oov=oov)

#binary = prepare_dataset.convert_to_binary_labels("data/", oov, language) #Ignore

In [ ]:
#Finds the labels 
all_labels = []
for i in os.listdir("data"):
    if os.path.isdir("data/"+i):
        for phase in os.listdir("data/"+i):
            for run in range(9):
                path = f"data/{i}/{run}/{language}/labels.txt"
                with open(path, "r") as f:
                    lines = f.readlines()
                    for l in lines:
                        all_labels.append(l.strip("\n"))

from collections import Counter
import codecs

most_common = Counter(all_labels).most_common()
label_dict = {}
with codecs.open("label_dict.txt", "r", encoding="utf-8") as f:
    for i in f.readlines():
        k = i.split("=")
        label_dict[k[1].strip("\n ")] = k[0]
most_common_lemmas = []
for i in range(len(most_common)):
    lbl = most_common[i][0]
    most_common_lemmas.append((label_dict[lbl], most_common[i][1]))



In [ ]:
vocab15 = []
off_set = 0 #=2 for ADJ, =1 for the rest (it is to exclude the most frequent words which leads to overfitting)
label_stats = most_common_lemmas[off_set+1:15+off_set+1]
for w, _ in label_stats: #top 15 words
    vocab15.append(w.strip())
    
print(vocab15)
print(label_stats)

In [ ]:
vocab = vocab15
prepare_dataset.prepare_handpicked_labels(annotation_file, "data/", vocab, oov=oov)

In [ ]:
#Exclude data that corresponds to oov
with codecs.open("label_dict.txt", "r", encoding="utf-8") as f:
    for i in f.readlines():
        k = i.split("=")
        if k[0].strip() == oov:
            label_2_remove = int(k[1].strip("\n "))
            break
print(label_2_remove)
labels_dict = {}
for i in os.listdir("data"):
    if os.path.isdir("data/"+i):
        for phase in os.listdir("data/"+i):
            for run in range(9):
                labels = np.loadtxt(f"data/{i}/{run}/{language}/labels.txt")
                labels_dict[run] = labels

exclusion_idxs = {k:[] for k in range(9)}

for run in labels_dict.keys():
    for idx, label in enumerate(labels_dict[run]):
        if label==label_2_remove:
            exclusion_idxs[run].append(idx)


for i in os.listdir("data"):
    if os.path.isdir("data/"+i):
        for phase in os.listdir("data/"+i):
            for run in range(9):
                path = f"data/{i}/{run}/{language}/labels.txt"
                with open(path, "r") as f:
                    lines = f.readlines()
                with open(path, "w") as f:
                    for line in lines:
                        if line.strip("\n ") != str(label_2_remove):
                            f.write(line)

count = 0
for phase in os.listdir("data/"):
    for run in range(9):
        path = f"data/{phase}/{run}/{language}/"
        for file in os.listdir(path):
            if file.endswith(".txt"):
                continue
            file_path = f"data/{phase}/{run}/{language}/{file}"
            idx = file.split("_")[-1][:-4]
            if int(idx) in exclusion_idxs[run]:
                count +=1 
                dest = f"aux_data/{phase}/{run}/{language}/"
                if not os.path.exists(dest):
                    os.makedirs(dest)
                shutil.move(file_path, f"aux_data/{phase}/{run}/{language}/")
                #time.sleep(0.00001)
print("moved ", count, " files")


In [ ]:

def count_files():
    counts = {}
    # Iterate directory
    for phase in os.listdir("data"):
        if os.path.isdir("data/"+phase):
            count=0
            for run in os.listdir(f"data/{phase}/"):
                for file in os.listdir(f"data/{phase}/{run}/{language}"):
                    if file.endswith(".npy"):
                        count += 1
        counts[phase] = count
    return counts

count_files()

In [ ]:
def calc_baseline(lbl = "data/Test/", language = "EN"):
    prob = 0
    labels = []
    for run in os.listdir(lbl):
        path = lbl +"/"+ run + "/" + language
        multiplier = len(os.listdir(path))-1
        if multiplier>0:
            for _ in range(int(multiplier/len(np.loadtxt(path+"/labels.txt")))):
                labels.append(np.loadtxt(path+"/labels.txt"))
    labels = np.concatenate(labels).ravel()
    total = labels.size
    label_c = Counter(labels)
    for _, v in label_c.items():
        prob += (v/total)**2
    return prob

calc_baseline(language=language)

In [ ]:
#Write a log
import json
import datetime
with codecs.open("config_stats.txt", "a", encoding="utf-8") as f:
    f.write("\n")
    f.write(f"{datetime.datetime.now()}\n")
    f.write(f"{off_set+1}th to {off_set+15}th\n")
    for i in label_stats:
        f.write(str(i) + ", ")
    f.write("\n")
    f.write(json.dumps(count_files()))
    f.write("\n")
f.close()

In [4]:
batch_size=50
config = f"{config_version}_{language}_top15{pos}"

In [5]:

train_pytorch.train(binary=False, batch_size = batch_size, num_epochs = 10, language=language, config=config, model = "2d", weights=None)

2022-12-12 21:46:28.015507


c:\Users\kjart\OneDrive\Dokumenter\KU\3. semester\Cog sci 3\CompCogSci3\mt_deep-master\scripts\model_m2dcnn.py:78: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(m.weight, gain=nn.init.calculate_gain('relu'))


cuda
Epoch 1/10
-------------
{'train': <torch.utils.data.dataloader.DataLoader object at 0x0000015352E1B550>, 'valid': <torch.utils.data.dataloader.DataLoader object at 0x0000015352E1B4C0>, 'test': <torch.utils.data.dataloader.DataLoader object at 0x0000015352E1AAA0>}


RuntimeError: CUDA error: CUBLAS_STATUS_EXECUTION_FAILED when calling `cublasSgemm( handle, opa, opb, m, n, k, &alpha, a, lda, b, ldb, &beta, c, ldc)`

In [ ]:
import model_m2dcnn as model
from torch.utils.data import DataLoader
from dataset import lpp_Dataset

batch_size = 50
binary = False
test_dataloader = DataLoader(lpp_Dataset("data/Test/"), batch_size=batch_size, shuffle=False)
if binary:
        nb_classes = 2
else:
    with open("label_dict.txt", "r") as f:
        nb_classes = len(f.readlines())
      
print(nb_classes)
train_pytorch.test(model.M2DCNN(numClass=nb_classes, numFeatues=30880, DIMX=74, DIMY=90, DIMZ=73), config,f"results/{config}_weights.pth", nb_classes, test_dataloader)